In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('./data/Symptom-severity.csv')


In [5]:
symptoms = list(df['Symptom'])

In [6]:
with open("./data/symptoms.txt", 'w') as f:
    for sym in symptoms:
        f.write(sym+"\n")


In [7]:
df = pd.read_csv('./data/symptom_Description.csv')

In [8]:
diseases = list(df['Disease'])

In [9]:
with open("./data/diseases.txt", 'w') as f:
    for dis in diseases:
        f.write(dis+"\n")


In [156]:
import pandas as pd
import numpy as np

def generate_data(csv_path, num_rows=None):
    df = pd.read_csv(csv_path)
    # set num_rows generated to same size as input df
    if not num_rows:
        num_rows = df.shape[0]
    # split into symptom and disease dfs
    irregular_symptoms_all = (df.groupby('Disease').sum() == 0)
    symptoms = df.iloc[:, 1:]
    diseases = pd.get_dummies(df['Disease'])
    # generate data
    gen_irr = generate_irregulars(symptoms, diseases, num_rows // 2, 
                                  irregular_symptoms_all, df)
    gen_reg = generate_regulars(symptoms, diseases, num_rows // 2)
    reg_irreg = pd.concat([gen_reg, gen_irr])
    return reg_irreg
    
def generate_irregulars(symptoms, diseases, num_rows, irregular_symptoms_all, df):
    irregular_indices = np.random.choice(
        range(diseases.shape[0]), size=num_rows)
    
    result = []
    for index in irregular_indices:
        reg = symptoms.iloc[index]
        irr = symptoms.columns[~reg.astype(bool)]
        disease = df['Disease'][index]
        while True:
            irr_symptom = np.random.choice(irr)
            if (irregular_symptoms_all.loc[disease, irr_symptom]):
                reg[irr_symptom] = 1
                break
            else:
                continue
        result.append(reg)
    gen_irr_symptoms = pd.concat(result, axis=1).T

    sampled_diseases = diseases.loc[irregular_indices]
    gen_irr_symptoms = pd.concat([sampled_diseases, gen_irr_symptoms], axis=1)
    gen_irr_symptoms['Irregular'] = 1
    return gen_irr_symptoms

def generate_regulars(symptoms, diseases, num_rows):
    gen_reg_symptoms = diseases.sample(num_rows, replace=True)
    sampled_symptoms = symptoms.loc[gen_reg_symptoms.index]
    gen_reg_symptoms = pd.concat([gen_reg_symptoms, sampled_symptoms], axis=1)
    gen_reg_symptoms['Irregular'] = 0
    return gen_reg_symptoms

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from dataset import * 

class SymptomCheckerModel():
    def __init__(self):
        self.model = LogisticRegression()

    def fit(self, reg_irreg):
        X = reg_irreg.iloc[:, :-1]
        y = reg_irreg['irregular']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        self.model.fit(X_train, y_train)
        return self.model, self.model.score(X_test, y_test)

    def predict(self, x, thresh=0.4):
        result = self.model.predict_proba(x)
        irregular = result[:, 1] > thresh
        return result, irregular


In [92]:
df = generate_data("data/one_hot.csv")
model = SymptomCheckerModel()
model.fit(df)

df1 = df.iloc[:, :41]
df1.columns = df1.columns.str.replace(" (disease)", "").str.replace("(vertigo)", "vertigo")
df1.columns


diseasesDictionary = {
  "fungal_infection": 1,
  "allergy": 2,
  "gerd": 3,
  "chronic_cholestasis": 4,
  "drug_reaction": 5,
  "peptic_ulcer_disease": 6,
  "aids": 7,
  "diabetes": 8,
  "gastroenteritis": 9,
  "bronchial_asthma": 10,
  "hypertension": 11,
  "migraine": 12,
  "cervical_spondylosis": 13,
  "paralysis_(brain_hemorrhage)": 14,
  "jaundice": 15,
  "malaria": 16,
  "chicken_pox": 17,
  "dengue": 18,
  "typhoid": 19,
  "hepatitis_a": 20,
  "hepatitis_b": 21,
  "hepatitis_c": 22,
  "hepatitis_d": 23,
  "hepatitis_e": 24,
  "alcoholic_hepatitis": 25,
  "tuberculosis": 26,
  "common_cold": 27,
  "pneumonia": 28,
  "dimorphic_hemmorhoids(piles)": 29,
  "heart_attack": 30,
  "varicose_veins": 31,
  "hypothyroidism": 32,
  "hyperthyroidism": 33,
  "hypoglycemia": 34,
  "osteoarthristis": 35,
  "arthritis": 36,
  "vertigo_paroymsal_positional_vertigo": 37,
  "acne": 38,
  "urinary_tract_infection": 39,
  "psoriasis": 40,
  "impetigo": 41
}
df1.columns = df1.columns.str.lower().str.replace("__", "_")
a = [i for i in diseasesDictionary.keys()]
df1.columns = df1.columns.str.replace("  ", "_").str.lower().str.replace(" ", "_").str.lower()
df1.columns = [i[:-1] if i[-1] == "_" else i for i in df1.columns]
df1

/Users/cirozhang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,vertigo_paroymsal_positional_vertigo,aids,acne,alcoholic_hepatitis,allergy,arthritis,bronchial_asthma,cervical_spondylosis,chicken_pox,chronic_cholestasis,...,osteoarthristis,paralysis_(brain_hemorrhage),peptic_ulcer_diseae,pneumonia,psoriasis,tuberculosis,typhoid,urinary_tract_infection,varicose_veins,hepatitis_a
1648,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1468,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
429,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1878,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3878,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4504,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1097,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2346,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4154,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [100]:
df1 = df1.rename(columns={'peptic_ulcer_diseae': 'peptic_ulcer_disease'})

df1.columns

Index(['vertigo_paroymsal_positional_vertigo', 'aids', 'acne',
       'alcoholic_hepatitis', 'allergy', 'arthritis', 'bronchial_asthma',
       'cervical_spondylosis', 'chicken_pox', 'chronic_cholestasis',
       'common_cold', 'dengue', 'diabetes', 'dimorphic_hemmorhoids(piles)',
       'drug_reaction', 'fungal_infection', 'gerd', 'gastroenteritis',
       'heart_attack', 'hepatitis_b', 'hepatitis_c', 'hepatitis_d',
       'hepatitis_e', 'hypertension', 'hyperthyroidism', 'hypoglycemia',
       'hypothyroidism', 'impetigo', 'jaundice', 'malaria', 'migraine',
       'osteoarthristis', 'paralysis_(brain_hemorrhage)',
       'peptic_ulcer_disease', 'pneumonia', 'psoriasis', 'tuberculosis',
       'typhoid', 'urinary_tract_infection', 'varicose_veins', 'hepatitis_a'],
      dtype='object')

In [102]:
df1[a].astype(int)

,fungal_infection,allergy,gerd,chronic_cholestasis,drug_reaction,peptic_ulcer_disease,aids,diabetes,gastroenteritis,bronchial_asthma,...,hypothyroidism,hyperthyroidism,hypoglycemia,osteoarthristis,arthritis,vertigo_paroymsal_positional_vertigo,acne,urinary_tract_infection,psoriasis,impetigo
1648,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1468,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
429,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1878,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3878,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4504,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
df2 = df.iloc[:, 41:]
df2.columns = df2.columns.str.replace("_(disease)", "").str.lower().str.replace("(vertigo)", "vertigo").str.replace(' ', '_').str.replace('__', '_')
df2

,vertigo_paroymsal_positional_vertigo,abdominal_pain,abnormal_menstruation,acidity,acne,acute_liver_failure,aids,alcoholic_hepatitis,allergy,altered_sensorium,...,watering_from_eyes,weakness_in_limbs,weakness_of_one_body_side,weight_gain,weight_loss,yellow_crust_ooze,yellow_urine,yellowing_of_eyes,yellowish_skin,irregular
1648,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1468,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
429,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
1878,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3878,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4504,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4154,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1


In [50]:
weights = model.model.coef_  
intercept = model.model.intercept_
np.concatenate((intercept, weights[0]))

array([-6.32433824,  0.07931535,  0.31380747,  1.03081918, -0.75801071,
        1.1593566 , -0.05386011,  1.39792247, -0.27380922,  0.44649045,
        0.64548639, -1.97011473, -1.32721095, -0.79828546, -0.70350608,
        0.49497192,  0.99676533,  0.61760799,  1.87973411,  1.92795445,
       -1.12955459,  0.58968364,  0.82579867, -2.04536558, -0.1724984 ,
       -0.62014531, -1.97178027, -1.62410795,  0.56607831,  0.93987923,
        0.47088208, -0.61922034,  0.08239496,  1.37522652,  0.03377015,
       -0.02013016,  0.37491984, -1.12247282, -0.42340805, -0.11222528,
       -0.33785035, -0.27227225,  1.43859012,  0.47439524,  0.61695993,
        0.52318389,  1.52704018,  0.70600608,  1.55029181,  0.63483065,
        1.15828349,  0.83898076,  0.93299985,  0.35923476,  0.61689293,
        0.3412288 ,  0.57369891,  1.06568407,  0.64616032,  0.17866374,
        0.74822072,  0.34984351,  0.81882826,  0.74584959,  1.06584182,
        0.48341724,  1.51437323,  1.21108424,  0.33025132,  1.22

In [63]:
model.predict([np.zeros(df.shape[1]-1)])

/Users/cirozhang/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


(array([[0.99821105, 0.00178895]]), array([False]))

In [81]:

input = df.iloc[-1].astype(bool)
# model.predict([input])


df.columns[input]

Index(['Hypertension ', 'Chest Pain', 'Dizziness', 'Headache', 'Hypertension',
       'Lack Of Concentration', 'Loss Of Balance', 'Small Dents In Nails',
       'Irregular'],
      dtype='object')

In [24]:
df.iloc[0]

,(vertigo) Paroymsal Positional Vertigo,AIDS,Acne,Alcoholic hepatitis,Allergy,Arthritis,Bronchial Asthma,Cervical spondylosis,Chicken pox,Chronic cholestasis,...,Watering From Eyes,Weakness In Limbs,Weakness Of One Body Side,Weight Gain,Weight Loss,Yellow Crust Ooze,Yellow Urine,Yellowing Of Eyes,Yellowish Skin,Irregular
3303,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,1,1,0
3078,False,False,False,False,False,False,False,False,False,True,...,0,0,0,0,0,0,0,1,1,0
3650,False,False,False,False,True,False,False,False,False,False,...,1,0,0,0,0,0,0,0,0,0
1082,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
4529,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1827,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,1
4789,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,1,0,0,0,0,1
4254,False,False,False,False,False,False,False,False,False,False,...,0,0,0,1,0,0,0,0,0,1
4420,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,1


In [84]:
len(df.columns)

214

In [85]:
len(set(df.columns))

190

In [107]:
df1

,vertigo_paroymsal_positional_vertigo,aids,acne,alcoholic_hepatitis,allergy,arthritis,bronchial_asthma,cervical_spondylosis,chicken_pox,chronic_cholestasis,...,osteoarthristis,paralysis_(brain_hemorrhage),peptic_ulcer_disease,pneumonia,psoriasis,tuberculosis,typhoid,urinary_tract_infection,varicose_veins,hepatitis_a
1648,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1468,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
429,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1878,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3878,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4504,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1097,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2346,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4154,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [119]:
df2.columns

Index(['vertigo_paroymsal_positional_vertigo', 'abdominal_pain',
       'abnormal_menstruation', 'acidity', 'acne', 'acute_liver_failure',
       'aids', 'alcoholic_hepatitis', 'allergy', 'altered_sensorium',
       ...
       'watering_from_eyes', 'weakness_in_limbs', 'weakness_of_one_body_side',
       'weight_gain', 'weight_loss', 'yellow_crust_ooze', 'yellow_urine',
       'yellowing_of_eyes', 'yellowish_skin', 'irregular'],
      dtype='object', length=173)

In [139]:
df2 = df2.rename(columns={'dimorphic_hemmorhoids(piles)':'dimorphic_hemorrhoids(piles)'})

In [145]:
df1 = df1.astype(int)

In [147]:
df = pd.concat([df1, df2], axis=1)


,vertigo_paroymsal_positional_vertigo,aids,acne,alcoholic_hepatitis,allergy,arthritis,bronchial_asthma,cervical_spondylosis,chicken_pox,chronic_cholestasis,...,watering_from_eyes,weakness_in_limbs,weakness_of_one_body_side,weight_gain,weight_loss,yellow_crust_ooze,yellow_urine,yellowing_of_eyes,yellowish_skin,irregular
1648,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1468,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
429,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1878,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3878,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4504,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4154,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [112]:
symptomsDictionary = {
        "vertigo_paroxysmal_positional_vertigo": 1,
        "abdominal_pain": 2,
        "abnormal_menstruation": 3,
        "acidity": 4,
        "acne": 5,
        "acute_liver_failure": 6,
        "aids": 7,
        "alcoholic_hepatitis": 8,
        "allergy": 9,
        "altered_sensorium": 10,
        "anxiety": 11,
        "arthritis": 12,
        "back_pain": 13,
        "belly_pain": 14,
        "blackheads": 15,
        "bladder_discomfort": 16,
        "blister": 17,
        "blood_in_sputum": 18,
        "bloody_stool": 19,
        "blurred_and_distorted_vision": 20,
        "breathlessness": 21,
        "brittle_nails": 22,
        "bronchial_asthma": 23,
        "bruising": 24,
        "burning_micturition": 25,
        "cervical_spondylosis": 26,
        "chest_pain": 27,
        "chicken_pox": 28,
        "chills": 29,
        "chronic_cholestasis": 30,
        "cold_hands_and_feets": 31,
        "coma": 32,
        "common_cold": 33,
        "congestion": 34,
        "constipation": 35,
        "continuous_feel_of_urine": 36,
        "continuous_sneezing": 37,
        "cough": 38,
        "cramps": 39,
        "dark_urine": 40,
        "dehydration": 41,
        "dengue": 42,
        "depression": 43,
        "diabetes": 44,
        "diarrhoea": 45,
        "dimorphic_hemorrhoids(piles)": 46,
        "dischromic_patches": 47,
        "distention_of_abdomen": 48,
        "dizziness": 49,
        "drug_reaction": 50,
        "drying_and_tingling_lips": 51,
        "enlarged_thyroid": 52,
        "excessive_hunger": 53,
        "extra_marital_contacts": 54,
        "family_history": 55,
        "fast_heart_rate": 56,
        "fatigue": 57,
        "fluid_overload": 58,
        "foul_smell_of_urine": 59,
        "fungal_infection": 60,
        "gastroenteritis": 61,
        "gerd": 62,
        "headache": 63,
        "heart_attack": 64,
        "hepatitis_a": 65,
        "hepatitis_b": 66,
        "hepatitis_c": 67,
        "hepatitis_d": 68,
        "hepatitis_e": 69,
        "high_fever": 70,
        "hip_joint_pain": 71,
        "history_of_alcohol_consumption": 72,
        "hypertension": 73,
        "hyperthyroidism": 74,
        "hypoglycemia": 75,
        "hypothyroidism": 76,
        "impetigo": 77,
        "increased_appetite": 78,
        "indigestion": 79,
        "inflammatory_nails": 80,
        "internal_itching": 81,
        "irregular_sugar_level": 82,
        "irritability": 83,
        "irritation_in_anus": 84,
        "itching": 85,
        "jaundice": 86,
        "joint_pain": 87,
        "knee_pain": 88,
        "lack_of_concentration": 89,
        "lethargy": 90,
        "loss_of_appetite": 91,
        "loss_of_balance": 92,
        "loss_of_smell": 93,
        "malaise": 94,
        "malaria": 95,
        "migraine": 96,
        "mild_fever": 97,
        "mood_swings": 98,
        "movement_stiffness": 99,
        "mucoid_sputum": 100,
        "muscle_pain": 101,
        "muscle_wasting": 102,
        "muscle_weakness": 103,
        "nausea": 104,
        "neck_pain": 105,
        "nodal_skin_eruptions": 106,
        "obesity": 107,
        "osteoarthristis": 108,
        "pain_behind_the_eyes": 109,
        "pain_during_bowel_movements": 110,
        "pain_in_anal_region": 111,
        "painful_walking": 112,
        "palpitations": 113,
        "paralysis_(brain_hemorrhage)": 114,
        "passage_of_gases": 115,
        "patches_in_throat": 116,
        "peptic_ulcer_disease": 117,
        "phlegm": 118,
        "pneumonia": 119,
        "polyuria": 120,
        "prominent_veins_on_calf": 121,
        "psoriasis": 122,
        "puffy_face_and_eyes": 123,
        "pus_filled_pimples": 124,
        "receiving_blood_transfusion": 125,
        "receiving_unsterile_injections": 126,
        "red_sore_around_nose": 127,
        "red_spots_over_body": 128,
        "redness_of_eyes": 129,
        "restlessness": 130,
        "runny_nose": 131,
        "rusty_sputum": 132,
        "scurring": 133,
        "shivering": 134,
        "silver_like_dusting": 135,
        "sinus_pressure": 136,
        "skin_peeling": 137,
        "skin_rash": 138,
        "slurred_speech": 139,
        "small_dents_in_nails": 140,
        "spinning_movements": 141,
        "spotting_urination": 142,
        "stiff_neck": 143,
        "stomach_bleeding": 144,
        "stomach_pain": 145,
        "sunken_eyes": 146,
        "sweating": 147,
        "swelled_lymph_nodes": 148,
        "swelling_joints": 149,
        "swelling_of_stomach": 150,
        "swollen_blood_vessels": 151,
        "swollen_extremeties": 152,
        "swollen_legs": 153,
        "throat_irritation": 154,
        "toxic_look_(typhos)": 155,
        "tuberculosis": 156,
        "typhoid": 157,
        "ulcers_on_tongue": 158,
        "unsteadiness": 159,
        "urinary_tract_infection": 160,
        "varicose_veins": 161,
        "visual_disturbances": 162,
        "vomiting": 163,
        "watering_from_eyes": 164,
        "weakness_in_limbs": 165,
        "weakness_of_one_body_side": 166,
        "weight_gain": 167,
        "weight_loss": 168,
        "yellow_crust_ooze": 169,
        "yellow_urine": 170,
        "yellowing_of_eyes": 171,
        "yellowish_skin": 172
};
x = list(symptomsDictionary .keys())


['vertigo_paroxysmal_positional_vertigo',
 'abdominal_pain',
 'abnormal_menstruation',
 'acidity',
 'acne',
 'acute_liver_failure',
 'aids',
 'alcoholic_hepatitis',
 'allergy',
 'altered_sensorium',
 'anxiety',
 'arthritis',
 'back_pain',
 'belly_pain',
 'blackheads',
 'bladder_discomfort',
 'blister',
 'blood_in_sputum',
 'bloody_stool',
 'blurred_and_distorted_vision',
 'breathlessness',
 'brittle_nails',
 'bronchial_asthma',
 'bruising',
 'burning_micturition',
 'cervical_spondylosis',
 'chest_pain',
 'chicken_pox',
 'chills',
 'chronic_cholestasis',
 'cold_hands_and_feets',
 'coma',
 'common_cold',
 'congestion',
 'constipation',
 'continuous_feel_of_urine',
 'continuous_sneezing',
 'cough',
 'cramps',
 'dark_urine',
 'dehydration',
 'dengue',
 'depression',
 'diabetes',
 'diarrhoea',
 'dimorphic_hemorrhoids(piles)',
 'dischromic_patches',
 'distention_of_abdomen',
 'dizziness',
 'drug_reaction',
 'drying_and_tingling_lips',
 'enlarged_thyroid',
 'excessive_hunger',
 'extra_marital

In [121]:
df2.columns = [i[:-1] if i[-1] == "_" else i for i in df2.columns]

In [152]:
X = df.drop(['irregular'], axis=1)
y = df['irregular']

In [162]:
model = SymptomCheckerModel()
model.fit(df)

/Users/cirozhang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(LogisticRegression(), 0.717479674796748)

In [165]:
w = model.model.coef_

In [166]:
b = model.model.intercept_

In [170]:
np.concatenate((b, w[0]))

array([-6.17685481e+00, -2.77170405e-01,  8.93156566e-01,  2.43742015e-01,
       -3.96570238e-02,  1.17330057e+00,  6.76264452e-01,  1.02111384e+00,
        5.61776927e-01,  5.62137511e-01,  3.24273037e-01, -2.87935646e+00,
       -1.12410923e+00, -1.08534867e+00, -7.84726109e-01,  1.11290747e+00,
        3.36492223e-01,  2.38130135e-01,  9.86841534e-01,  1.65308316e+00,
       -9.11262864e-01,  1.61280757e+00,  1.04524228e+00, -9.32859538e-01,
        5.08023295e-01, -1.09689955e+00, -4.68210739e-01, -2.23651914e+00,
       -3.27193921e-02,  5.21347565e-01,  8.56712737e-01, -2.23049261e-01,
       -1.99303277e-01,  1.03905912e+00,  5.31353689e-01,  1.84435061e-02,
       -6.05005972e-01, -1.60302078e+00, -6.12699919e-01,  4.84247633e-01,
       -4.46653648e-01, -8.69734486e-01,  1.48850225e+00,  2.73577572e-01,
        5.74977863e-01,  8.80251882e-01,  1.36362890e+00,  9.63506028e-01,
        1.33759276e+00,  1.15808328e+00,  1.90050008e+00,  1.31693760e+00,
        5.31408117e-01,  